In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import numpy as np
%matplotlib inline

In [5]:
data = pd.read_excel('/home/dariodg/Desktop/free/Galpao/ComissionApp/docs/comissabril2019.XLS')

In [8]:
data.columns

Index(['turma', 'titulo', 'vencimento', 'sacado', 'recebido', 'credito',
       'descritivo'],
      dtype='object')

## Turmas - Adicionando "sem turma" aos valores vazios

In [11]:
turmas = data['turma']

In [12]:
turmas.describe()

count                          218
unique                          35
top       2019 AÉREOS 2x - TURMA 3
freq                            18
Name: turma, dtype: object

In [39]:
turmas[type(turmas) == float]

KeyError: False

In [37]:
print(type(turmas[2]) == float)

False


In [44]:
def fill_sem_turma(value):
    if type(value) == float:
        return 'sem_turma'
    return value

In [45]:
turma2 = turmas.apply(fill_sem_turma)

In [46]:
turma2.head()

0                      sem_turma
1    2019 AC - ADOLESC - TURMA 1
2    2019 AC - ADOLESC - TURMA 1
3    2019 AC - ADOLESC - TURMA 1
4    2019 AC - ADOLESC - TURMA 1
Name: turma, dtype: object